In [2]:
import pandas as pd
import matplotlib.pyplot as plt
# import akshare as ak
import datetime
import glob
import numpy as np

In [3]:
a = np.array([1,2,3,4])


In [6]:
a = pd.DataFrame(a)

In [8]:
a.shift(2)

,0
0,NaN
1,NaN
2,1.0
3,2.0


In [3]:
def get_data(index="sh510050"):
    sh50_files = sorted(glob.glob("../sh_index_data_tdq/{}_*.tsv".format(index)))

    sh50_pd = pd.read_table(sh50_files[0], header=0, index_col=0,parse_dates=["day"])
    for f in sh50_files[1:]:
        add_pd = pd.read_table(f, header=0, index_col=0,parse_dates=["day"])
        add_pd = add_pd[add_pd["day"] > list(sh50_pd["day"])[-1]] 
        sh50_pd = sh50_pd.append(add_pd)
        
    if index == "sh512100":
        sh50_pd.loc[sh50_pd["datetime"].map(str) <"2022-09-05", "close"] *= 2.6895

    sh50_pd.reset_index(drop=True, inplace=True)
    sh50_pd["close_yes"] = sh50_pd["close"].shift()
    sh50_pd = sh50_pd[sh50_pd["datetime"].map(str) > "2021-03-18"]
    return sh50_pd

In [9]:
a[::2]

,0
0,1
2,3


In [4]:
sh50_pd = get_data()

In [7]:
sh50_pd["mean5"] = sh50_pd["close"].rolling(5, min_periods=1).mean()

In [8]:
sh50_pd

,open,close,high,low,vol,amount,year,month,day,hour,minute,datetime,close_yes,mean5
240,3.580,3.589,3.598,3.580,27276800.0,97960312.0,2021,3,18,9,35,2021-03-18 09:35,3.576,3.589000
241,3.588,3.593,3.599,3.588,19561000.0,70313608.0,2021,3,18,9,40,2021-03-18 09:40,3.589,3.591000
242,3.593,3.587,3.595,3.583,9604000.0,34471792.0,2021,3,18,9,45,2021-03-18 09:45,3.593,3.589667
243,3.586,3.596,3.597,3.585,11333100.0,40702032.0,2021,3,18,9,50,2021-03-18 09:50,3.587,3.591250
244,3.596,3.595,3.597,3.592,8044000.0,28912864.0,2021,3,18,9,55,2021-03-18 09:55,3.596,3.592000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23563,2.630,2.632,2.632,2.626,16558900.0,43544520.0,2023,3,17,14,40,2023-03-17 14:40,2.630,2.631400
23564,2.632,2.632,2.634,2.631,12506800.0,32918140.0,2023,3,17,14,45,2023-03-17 14:45,2.632,2.630800
23565,2.633,2.635,2.636,2.632,10827300.0,28514916.0,2023,3,17,14,50,2023-03-17 14:50,2.632,2.631600
23566,2.635,2.638,2.638,2.634,17295600.0,45572832.0,2023,3,17,14,55,2023-03-17 14:55,2.635,2.633400


In [91]:
def split_data(stock, lookback): 
    data_raw=stock.to_numpy() 
    data=[]
    #you can free play(seg_length)
    for index in range(len(data_raw)-lookback): 
        data.append(data_raw[index: index + lookback]) 
    data=np.array(data)

    test_set_size=int(np.round(0.2*data.shape[0])) 
    train_set_size=data.shape[0]-(test_set_size) 
    x_train=data[:train_set_size,:-48]

    y_train=data[:train_set_size,[-43, -37, -25, -1]]
#     y_train12=data[:train_set_size,-37]
#     y_train24=data[:train_set_size,-25]
#     y_train48=data[:train_set_size,-1]
   
    x_test=data[train_set_size:,:-48] 
    y_test=data[train_set_size:,[-43, -37, -25, -1]]

    return [x_train,y_train,  x_test,y_test] 

In [92]:
x_train,y_train, x_test,y_test = split_data(sh50_pd["mean5"], 480)

In [57]:
y_train

array([[3.527 , 3.519 , 3.5184, 3.5242],
       [3.5238, 3.5168, 3.5188, 3.5268],
       [3.5224, 3.516 , 3.5186, 3.5302],
       ...,
       [2.434 , 2.4358, 2.4606, 2.4338],
       [2.4346, 2.437 , 2.4578, 2.4354],
       [2.434 , 2.4398, 2.4552, 2.4346]])

In [20]:
np.save("sh510050_train.npy", x_train)

In [93]:
np.save("sh510050_test.npy", x_test)

In [22]:
x_train

array([[3.589     , 3.591     , 3.58966667, ..., 3.553     , 3.554     ,
        3.5558    ],
       [3.591     , 3.58966667, 3.59125   , ..., 3.554     , 3.5558    ,
        3.554     ],
       [3.58966667, 3.59125   , 3.592     , ..., 3.5558    , 3.554     ,
        3.5502    ],
       ...,
       [2.5726    , 2.5712    , 2.5696    , ..., 2.4446    , 2.4434    ,
        2.4412    ],
       [2.5712    , 2.5696    , 2.5682    , ..., 2.4434    , 2.4412    ,
        2.439     ],
       [2.5696    , 2.5682    , 2.567     , ..., 2.4412    , 2.439     ,
        2.437     ]])

In [32]:
def normal_by_row(data):

    # 计算每列的均值和标准差
    mean = np.mean(data, axis=1, keepdims=True)
    std = np.std(data, axis=1, keepdims=True)

    # 标准化数据
    data_standardized = (data - mean) / std
    return data_standardized



In [33]:
x_train_nor = normal_by_row(x_train)

In [30]:
data_standardized

array([[ 1.07207866, -1.33477928,  0.26270062],
       [-0.92124924, -0.46860887,  1.38985811],
       [ 1.13892604,  0.15657804, -1.29550407],
       [-1.27308629,  1.16987088,  0.10321542],
       [-1.41350423,  0.66796623,  0.745538  ]])

In [35]:
len(x_train_nor)

18278

In [36]:
len(y_train)

18278

In [59]:
las = x_train[:, [-1, -1, -1, -1]]

In [39]:
y_train

array([3.5242, 3.5268, 3.5302, ..., 2.4338, 2.4354, 2.4346])

In [60]:
pct_change = (y_train - las)/las

In [94]:
lastest = x_test[:, [-1, -1, -1, -1]]
pct_change_test = (y_test - lastest)/lastest

In [44]:
len(pct_change)

18278

In [46]:
max(pct_change)

0.07034390352836088

In [62]:
np.save("sh510050_pct_change.npy", pct_change)

In [95]:
np.save("sh510050_pct_change_test.npy", pct_change_test)

In [53]:
y_train=data[:train_set_size,-42]

NameError: name 'train_set_size' is not defined

In [63]:
def vol_normal(data):
    a= np.array(data)
    dates = int(len(a)/48)
    a = a.reshape(dates, 48)
    avg = np.average(a, axis=0)
    meana = np.tile(avg, dates)
    normal_data = data/meana
    return normal_data

In [64]:
sh50_pd["vol_norm"] = vol_normal(sh50_pd["vol"])

In [77]:
sh50_pd["vol_norm_mean5"] = sh50_pd["vol_norm"].rolling(12, min_periods=1).mean()

In [80]:
len(sh50_pd)

23328

In [96]:
x_train_vol,y_train_vol, x_test_vol,y_test_vol = split_data(sh50_pd["vol_norm_mean5"], 480)

In [81]:
len(x_train_vol)


18278

In [79]:
np.save("sh510050_train_vol.npy", x_train_vol)

In [97]:
np.save("sh510050_test_vol.npy", x_test_vol)

In [83]:
18278+ 4570

22848

In [84]:
23328 - 22848

480

In [89]:
ch50_pd_choose =  sh50_pd[480:18278 + 480]

In [90]:
ch50_pd_choose.to_csv("ch50_choose",  sep="\t", header=True)

In [ ]:
lench50_pd_choose